<a href="https://colab.research.google.com/github/mareloraby/ana-albahr/blob/main/MarkovChains-generate-based-on-given-Qafiya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Environment

In [ ]:
!pip install -q markovify
!pip install -q pyarabic

In [ ]:
!git clone https://github.com/mareloraby/ana-albahr.git

fatal: destination path 'ana-albahr' already exists and is not an empty directory.


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# poemsdf = pd.read_csv("/content/drive/MyDrive/NLP/PoetryGeneration/Arabic_poetry_dataset.csv",  encoding='utf-8') 
# cats = ['العصر العباسي', 'العصر المملوكي', 'العصر الايوبي', 'العصر العثماني', 'العصر الاموي', 'العصر الأندلسي', 'عصر المخضرمون', 'العصر الجاهلي', 'العصر الإسلامي']
# poemsdf2 = poemsdf[poemsdf['category'].isin(cats)].reset_index()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PoemsFile = "/content/drive/MyDrive/NLP/PoetryGeneration/datasetarabic.txt" #poems text file path
all_poems = open(PoemsFile, 'rb').read().decode(encoding='utf-8') 

In [ ]:
import pickle

#import a list of arabic words - resource: unknown

with open('/content/drive/MyDrive/NLP/PoetryGeneration/vocabs.pkl', 'rb') as pickle_load:
    voc_list = pickle.load(pickle_load)

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

#create a list of all words in the dataset 

tokenizer = Tokenizer()
corpus = all_poems.split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print('Total number of words in corpus:',total_words)

corpus_vocab = []
for key,val in enumerate(tokenizer.word_index):
  if(len(val) > 2):
    corpus_vocab.append(val)  

# append the list of all words in the dataset with the list of arabic words
big_vocab_list = (corpus_vocab+voc_list)
print('Total number of words in the current list: ',len(big_vocab_list))

Total number of words in corpus: 1054587
Total number of words in the current list:  1990649


In [ ]:
#make sure you only keep unique words
big_set = set()
for x in big_vocab_list:
    big_set.add(x)

In [ ]:
import pyarabic.araby as araby

#a function to remove Arabic diacritics (harakaat)

def remove_harakas(word):
  return araby.strip_diacritics(word)

In [ ]:
# testing the remove_harakas function

remove_harakas('حَيِّ الحُمولَ بِجانِبِ العَزلِ')

'حي الحمول بجانب العزل'

In [ ]:
# get all words that rhymes with the last 2 letters of the input
max_word_length = 9
def rhymes_with(word):
    if word not in ['الله', 'والله', 'بالله', 'لله', 'تالله']:
        word = word.replace('ّ', '')
    ending = word[-2:]
    # print(ending)
    rhymes = []

    for w in big_set: 
        if len(w) < max_word_length:

          if w.endswith(ending):
            rhymes.append(w)

          if(len(w) > 1 and len(ending) > 1 and w[-2] == ending[-2]):

            if ending.endswith('و') and w.endswith('ُ'):
              rhymes.append(w)

            if ending.endswith('ي') and (w.endswith('ِ')):
              rhymes.append(w)

            if ((ending.endswith('ا') or ending.endswith('ى')) and (w.endswith('َ') or w.endswith('ا') or w.endswith('ى') )):
              rhymes.append(w)
  

    return rhymes

In [ ]:
import markovify

# load Markovify, a simple, extensible Markov chain generator. src: github.com/jsvine/markovify

def markov(text_file):
    with open(text_file, 'r', encoding='utf-8') as f:
        text = f.read()
    text_model = markovify.NewlineText(text)
    return text_model

In [ ]:
text_model = markov(PoemsFile)

In [ ]:
from tqdm import tqdm

#generate poem lines based on a single rhyme.
def generate_poem_single_rhyme( rhyme, iterations=3000, use_tqdm=False):
    
    rhymes_list = rhymes_with(rhyme) # rhymes_with_last_n_chars(rhyme, n_of_rhyme_letters)
    bayts = set()
    used_rhymes = set()

    poem = ""
    
    if use_tqdm == True: 
        if hasattr(tqdm, '_instances'): tqdm._instances.clear()
        it_range = tqdm(range(iterations))
    else:
        it_range = range(iterations)


    for i in it_range:
        bayt = text_model.make_short_sentence(280, tries=100) # generate a line using markov chains
        last_word = bayt.split()[-1] # get last word of the generated line

        if (last_word in rhymes_list) and (last_word not in used_rhymes) and (bayt not in bayts):
            
            add_line = True
            
            temp = bayt.split(' ') 

            for word in temp:
              if len(word) == 1 and word!='و': # if the word has a length 1, check if it's not a و
                add_line = False
              if len(word) == 2 and len(remove_harakas(word)) == 1: # check length after removing harakas
                add_line = False
              if remove_harakas(word) == 'الله': #don't use lines that contains الله
                add_line = False
              if word == temp[-1] and word == rhyme:
                add_line = False
            
            if add_line:
              bayts.add(bayt)
              used_rhymes.add(last_word)
              poem += "{}\n".format(bayt)
              # if not use_tqdm:
              print(bayt)

    return poem

# Test

In [ ]:
rhyme = input('الرجاء إدخال القافية. مثال: ما \n(Enter) ثم انقر على\n')
print('القافية المختارة : ', rhyme)

if rhyme == '':
  print('الرجاء إدخال القافية. مثال: ما \n')
  print('ثم انقر على (Enter)')

else:
  x = generate_poem_single_rhyme(
                              rhyme,
                              iterations=2000, 
                              use_tqdm=True
                              )
  

الرجاء إدخال القافية. مثال: ما 
(Enter) ثم انقر على
مي
القافية المختارة :  مي


  6%|▌         | 112/2000 [00:05<01:31, 20.53it/s]

ساقٌ عَلى ساقٍ بِغَيرِ مَليمِ
بِهِ عِندَ الإِمامِ كَلامُهُم وَكَلامي


 15%|█▌        | 307/2000 [00:16<01:17, 21.85it/s]

وجد مضى عنها ولم يخِمِ


...

 83%|████████▎ | 1662/2000 [01:29<00:15, 21.65it/s]

شَفَت مِن دَهرِنا الحُطَمِ


 93%|█████████▎| 1867/2000 [01:40<00:06, 20.44it/s]

فالجسم منه في حَرَمِ


100%|██████████| 2000/2000 [01:47<00:00, 18.60it/s]


In [ ]:
print("The generated poem:\n",x)  

The generated poem:
 ساقٌ عَلى ساقٍ بِغَيرِ مَليمِ
بِهِ عِندَ الإِمامِ كَلامُهُم وَكَلامي
وجد مضى عنها ولم يخِمِ
وَمَكرُمَةٍ عِندَ السَماءِ جُسامِ
وامام الانشاء في كل يومٍ يا غلامي
كَأَنَّ دُفّاً عَلى عَلياءَ خالِدَةِ الأَرومِ
دو مِن خِلالِ غِمامِ
وَقالَ دَليلُ القَومِ قَد ذَهَبَ السَهمُ بِقَلبِ الرَمي
وحطَّم مَّران الوغى في الشكائمِ
رويداً على مضنًى من البينِ تَرْمي الشملَ منكم بما ترمي
وَرَمَيتَ هَضبَتَنا بِأَفوَقَ ناصِلٍ وَبِشَرِّ ذامِ
حكمت على الأعداء من كل مَوشيِّ الرقوم منمنَمِ
يا لها في كلّ وقت هامي
إِلى الصُبحِ لَم يَنجُ مِن رامِحٍ وَنابِلٍ وَرامِ
لَبَابُ سمعيَ عن عذل بل يا موردَ الأيتامِ
لَهُ مِن قَبيلِ السِهامِ
وَحَلَّت مَكاناً لَم يَكُن بِذَميمِ
كَم صُغت فيكَ مِن بِنتِ الكُرومِ
كُمونَ الموتِ في حَدِّ مَصقولِ الغِرارَيْنِ مِخْذَمِ
مصعَّدة من حيث لا تهتدي فيها الليوث لمجْثَمِ
صُنّا قَوائِمَها عَنهُم فَما مِن نَدىً وَدَمِ
فَكَم شاهدتُ قَبلك من نعيمٍ في النعامي
ولا برحتْ في القفرِ غيرَ البهائمِ
قُصورُ الوَرى عَن وَصلِها كَأَن لَمِ
رَأى العُذّالُ بَذلَ المالِ مِن خَيلٍ وَمِن نَعَ

src: 

https://github.com/hayderkharrufa/arabic_poem_generator/blob/master/generate_poems.ipynb